# Import 

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Data Loading 

In [13]:
# load slices subtracted by average slice
import os
import pickle
with open(os.getcwd() + "/data/sample/SampleData_Normal1.pkl", 'rb') as f :
    Data = pickle.load(f)
X_data = np.array(Data['X'])
y_data = np.array(Data['y'])
print "Loading Done"
config = tf.PGPUOptions(allow_growth=True, log_device_placement=True)
# sess = tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True)))
sess = tf.Session(config=config)
print "Session Created"

Loading Done


ValueError: Protocol message GPUOptions has no "log_device_placement" field.

In [3]:
# visualize slice


# Build Structure 

In [3]:
nOut = 13
nIn = 280*140
W = {"w1" : tf.Variable(tf.truncated_normal([5, 5, 3, 32], stddev=0.1)),
    "w2" : tf.Variable(tf.truncated_normal([5, 5, 32, 64], stddev=0.1)),
    "w3" : tf.Variable(tf.truncated_normal([14*7*64, 128], stddev=0.1)),
    "w4" : tf.Variable(tf.truncated_normal([128, nOut], stddev=0.1))}
B = {"b1" : tf.Variable(tf.random_normal([32], stddev=0.1)),
    "b2" : tf.Variable(tf.random_normal([64], stddev=0.1)),
    "b3" : tf.Variable(tf.random_normal([128], stddev=0.1)),
    "b4" : tf.Variable(tf.random_normal([nOut], stddev=0.1))}

x = tf.placeholder(tf.uint8, [None, 280, 140, 3])
y = tf.placeholder(tf.uint8, [None])
# X_data = tf.constant(X_train)
# y_data = tf.constant(y_train)
y_hot = tf.one_hot(y, nOut)
x_float = tf.cast(x, tf.float32)

# x_r = tf.reshape(x, [-1, 32, 32, 3]) 
conv1 = tf.nn.conv2d(x_float, W["w1"], strides=[1,1,1,1], padding='SAME')
mean, var = tf.nn.moments(conv1, [0,1,2])
conv1 = tf.nn.batch_normalization(conv1, mean, var, 0, 1, 0.001)
conv1 = tf.nn.relu(tf.nn.bias_add(conv1, B["b1"]))
pool1 = tf.nn.max_pool(conv1, ksize=[1,4,4,1], strides=[1,4,4,1], padding='SAME')

conv2 = tf.nn.conv2d(pool1, W["w2"], strides=[1,1,1,1], padding='SAME')
mean, var = tf.nn.moments(conv2, [0,1,2])
conv2 = tf.nn.batch_normalization(conv2, mean, var, 0, 1, 0.001)
conv2 = tf.nn.relu(tf.nn.bias_add(conv2, B["b2"]))
pool2 = tf.nn.max_pool(conv2, ksize=[1,5,5,1], strides=[1,5,5,1], padding='SAME')

fc1 = tf.reshape(pool2, [-1, W["w3"].get_shape().as_list()[0]])
fc1 = tf.nn.relu(tf.nn.bias_add(tf.matmul(fc1, W["w3"]), B["b3"]))

fc2 = tf.nn.bias_add(tf.matmul(fc1, W["w4"]), B["b4"])

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=fc2, labels=y_hot))
opt = tf.train.AdamOptimizer(learning_rate=0.01).minimize(loss)

In [6]:
corPred = tf.equal(tf.argmax(fc2, 1), tf.argmax(y_hot, 1))
acc = tf.reduce_mean(tf.cast(corPred, tf.float32))

# TensorBoard

In [10]:
# tf.summary.scalar('cross_entrophy', loss)
# tf.summary.scalar('accuracy', acc)
# merged = tf.summary.merge_all()
# summary_writer = tf.FWriter("/test", sess.graph)

# Run

In [12]:
n = len(X_data)
n_test = n/5
n_train = n - n_test
i = np.random.permutation(n)
X_train = X_data[i[:n_train]]
y_train = y_data[i[:n_train]]
X_test = X_data[i[n_train:]]
y_test = y_data[i[n_train:]]

trainEpochs = 50
dispStep = 2
batch_size = 50
batchNum = n_train/batch_size
# sess.run(tf.global_variables_initializer())
sess.run(tf.initialize_all_variables())

for epoch in range(trainEpochs):
    train_cost = 0
    train_acc = 0
    batchIdxs = np.random.permutation(n_train)
    for i in range(batchNum):
#         x_curr, y_curr = sess.run(x_batch, y_batch)
#         print y_curr
        x_batch = X_train[batchIdxs[i*batch_size:(i+1)*batch_size]]
        y_batch = y_train[batchIdxs[i*batch_size:(i+1)*batch_size]]
        sess.run(opt, feed_dict={x:x_batch, y:y_batch})
#         summary, _ = sess.run([merged, opt], feed_dict={x:x_batch, y:y_batch})
#         summary_writer.add_summary(summary, batchNum*epoch + i)
        train_cost += sess.run(loss, feed_dict={x:x_batch, y:y_batch})/batchNum
        train_acc += sess.run(acc, feed_dict={x:x_batch, y:y_batch})/batchNum
    if epoch%dispStep == 0:
#         train_cost = sess.run(loss, feed_dict={x:X_train, y:y_train})
#         train_acc = sess.run(acc, feed_dict={x:X_train, y:y_train})
        test_acc = sess.run(acc, feed_dict={x:X_test, y:y_test})
        print "Epoch: {:03d}/{:03d} Cost : {:.3f} TrainAcc : {:.3f} TestAcc : {:.3f}".format(epoch, trainEpochs, train_cost, train_acc, test_acc)

Epoch: 000/050 Cost : 126.061 TrainAcc : 0.373 TestAcc : 0.027
Epoch: 002/050 Cost : 47.931 TrainAcc : 0.187 TestAcc : 0.514
Epoch: 004/050 Cost : 31.411 TrainAcc : 0.107 TestAcc : 0.027
Epoch: 006/050 Cost : 13.862 TrainAcc : 0.393 TestAcc : 0.351
Epoch: 008/050 Cost : 5.411 TrainAcc : 0.407 TestAcc : 0.378


KeyboardInterrupt: 

In [10]:
sess.close()